## AI 뉴스 Vector DB 생성 시스템 

AI 뉴스 데이터를 Vector DB로 변환하여 저장하는 시스템입니다. 
LangChain과 FAISS를 활용하여 뉴스 데이터를 벡터화하고 효율적으로 검색할 수 있도록 구성되어 있습니다.

주요 기능:
1. JSON 형식의 뉴스 데이터 로드
2. 문서를 적절한 크기로 청크 분할
3. OpenAI Embedding을 통한 벡터화
4. FAISS를 이용한 벡터 저장소 생성
5. 진행 상황 로깅 및 모니터링

시스템 특징:
- 배치 처리를 통한 대용량 데이터 처리
- 진행상황 표시 (tqdm 프로그레스바)
- 에러 처리 및 로깅
- 환경 변수를 통한 설정 관리

실행하기 전에 필요한 환경변수:
- OPENAI_API_KEY: OpenAI API 키
- OPENAI_EMBEDDING_MODEL: 사용할 임베딩 모델명
- NEWS_FILE_PATH: 뉴스 데이터가 있는 디렉토리 경로
- VECTOR_STORE_NAME: 벡터 저장소 저장 경로

아래 코드를 실행하면 뉴스 데이터를 벡터 DB로 변환하여 저장합니다.

### 의미 기반 검색 시스템

In [1]:
import os
import json
import glob
from typing import List, Dict
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import JSONLoader
from langchain.schema import Document
from dotenv import load_dotenv
from tqdm import tqdm
import logging
import sys

class AINewsRAG:
    def __init__(self, embedding_model):
        self.embeddings = embedding_model
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        self.vector_store = None
        
        # 로깅 설정
        self.logger = logging.getLogger('AINewsRAG')
        # 기존 핸들러 제거
        if self.logger.handlers:
            self.logger.handlers.clear()
        
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler(sys.stdout)
        handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
        self.logger.addHandler(handler)
        # 로그 중복 방지
        self.logger.propagate = False
        
    def load_json_files(self, directory_path: str) -> List[Dict]:
        """여러 JSON 파일들을 로드합니다."""
        all_documents = []
        json_files = glob.glob(f"{directory_path}/ai_times_news_*.json")
        
        self.logger.info(f"총 {len(json_files)}개의 JSON 파일을 로드합니다...")
        
        for file_path in tqdm(json_files, desc="JSON 파일 로드 중"):
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    documents = json.load(file)
                    if documents :
                        documents = [doc for doc in documents if len(doc['content']) > 10]
                    #기사 내용이 없으면 생략
                    if len(documents) >= 10 : 
                        all_documents.extend(documents)
            except Exception as e:
                self.logger.error(f"파일 로드 중 오류 발생: {file_path} - {str(e)}")
        
        self.logger.info(f"총 {len(all_documents)}개의 뉴스 기사를 로드했습니다.")
        return all_documents
    
    def process_documents(self, documents: List[Dict]) -> List[Document]:
        """문서를 처리하고 청크로 분할합니다."""
        processed_docs = []
        self.logger.info("문서 처리 및 청크 분할을 시작합니다...")
        
        for doc in tqdm(documents, desc="문서 처리 중"):
            try:
                full_text = f"제목: {doc['title']}\n내용: {doc['content']}"
                metadata = {
                    'title': doc['title'],
                    'url': doc['url'],
                    'date': doc['date']
                }
                
                chunks = self.text_splitter.split_text(full_text)
                
                for chunk in chunks:
                    processed_docs.append(Document(
                        page_content=chunk,
                        metadata=metadata
                    ))
            except Exception as e:
                self.logger.error(f"문서 처리 중 오류 발생: {doc.get('title', 'Unknown')} - {str(e)}")
        
        self.logger.info(f"총 {len(processed_docs)}개의 청크가 생성되었습니다.")
        return processed_docs
    
    def create_vector_store(self, documents: List[Document]):
        """FAISS 벡터 스토어를 생성합니다."""
        self.logger.info("벡터 스토어 생성을 시작합니다...")
        total_docs = len(documents)
        
        try:
            # 청크를 더 작은 배치로 나누어 처리
            batch_size = 100
            for i in tqdm(range(0, total_docs, batch_size), desc="벡터 생성 중"):
                batch = documents[i:i+batch_size]
                if self.vector_store is None:
                    self.vector_store = FAISS.from_documents(batch, self.embeddings)
                else:
                    batch_vectorstore = FAISS.from_documents(batch, self.embeddings)
                    self.vector_store.merge_from(batch_vectorstore)
            
            self.logger.info("벡터 스토어 생성이 완료되었습니다.")
        except Exception as e:
            self.logger.error(f"벡터 스토어 생성 중 오류 발생: {str(e)}")
            raise
    
    def save_vector_store(self, path: str):
        """벡터 스토어를 로컬에 저장합니다."""
        try:
            self.logger.info(f"벡터 스토어를 {path}에 저장합니다...")
            self.vector_store.save_local(path)
            self.logger.info("벡터 스토어 저장이 완료되었습니다.")
        except Exception as e:
            self.logger.error(f"벡터 스토어 저장 중 오류 발생: {str(e)}")
            raise
    
    def load_vector_store(self, path: str):
        """저장된 벡터 스토어를 로드합니다."""
        try:
            self.logger.info(f"벡터 스토어를 {path}에서 로드합니다...")
            self.vector_store = FAISS.load_local(path, self.embeddings, allow_dangerous_deserialization =True)
            self.logger.info("벡터 스토어 로드가 완료되었습니다.")
        except Exception as e:
            self.logger.error(f"벡터 스토어 로드 중 오류 발생: {str(e)}")
            raise
    
    def search(self, query: str, k: int = 3) -> List[Document]:
        """쿼리와 관련된 문서를 검색합니다."""
        if self.vector_store is None:
            raise ValueError("Vector store has not been initialized")
        
        self.logger.info(f"'{query}' 검색을 시작합니다...")
        results = self.vector_store.similarity_search(query, k=k)
        self.logger.info(f"{len(results)}개의 관련 문서를 찾았습니다.")
        return results


### Vector DB 구축

In [4]:
# 사용 예시
def main():
    # 환경 변수 로드
    load_dotenv() 
    
    # 임베딩 모델
    embed_model = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
    
    # RAG 시스템 초기화
    rag_system = AINewsRAG(embed_model)
    
    # JSON 파일들 로드
    documents = rag_system.load_json_files(os.getenv("NEWS_FILE_PATH"))
    
    # 문서 처리
    processed_docs = rag_system.process_documents(documents)
    
    # 벡터 스토어 생성
    rag_system.create_vector_store(processed_docs)
    
    # 벡터 스토어 저장
    rag_system.save_vector_store(model=os.getenv("VECTOR_STORE_NAME"))
    
    # 검색 예시
    results = rag_system.search("RAG에 대한 뉴스")
    for doc in results:
        print(f"제목: {doc.metadata['title']}")
        print(f"내용: {doc.page_content[:200]}...")
        print(f"URL: {doc.metadata['url']}")
        print(f"날짜: {doc.metadata['date']}")
        print("-" * 80)

if __name__ == "__main__":
    main()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16020\4201626932.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embed_model = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))


ImportError: Could not import openai python package. Please install it with `pip install openai`.

In [ ]:
# 환경 변수 로드
load_dotenv() 

# 환경 변수에서 경로 가져오기
vector_store_path = os.getenv("VECTOR_STORE_NAME", "ai_news_vectorstore")

# 임베딩 모델
embed_model = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))

# RAG 시스템 초기화
rag = AINewsRAG(embed_model)


try:
    # 기존 벡터 스토어 로드 시도
    rag.load_vector_store(vector_store_path)
    print("✅ 기존 벡터 스토어를 로드했습니다.")
    
except Exception as e:
    print(f"벡터 스토어 로드 실패: {str(e)}")
    

#Query에 따른 문서 찾기

query = "AI 교육"


try:
    print(f"\n'{query}' 검색을 시작합니다...")
    
    results = rag.search(query, k=5)
    
    print(f"\n✨ 검색 완료! {len(results)}개의 결과를 찾았습니다.\n")
    
    # 결과 출력
    for i, doc in enumerate(results, 1):
        print(f"\n{'='*80}")
        print(f"검색 결과 {i}/{len(results)}")
        print(f"제목: {doc.metadata['title']}")
        print(f"날짜: {doc.metadata['date']}")
        print(f"{'-'*40}")
        print(f"URL: {doc.metadata['url']}")
        print(f"내용:\n{doc.page_content[:300]}...")
        
except Exception as e:
    print(f"\n❌ 검색 중 오류가 발생했습니다: {str(e)}")

2024-11-28 11:27:41,558 - 벡터 스토어를 ai_news_vectorstore에서 로드합니다...
2024-11-28 11:27:42,130 - 벡터 스토어 로드가 완료되었습니다.
✅ 기존 벡터 스토어를 로드했습니다.

'RAG 시스템' 검색을 시작합니다...
2024-11-28 11:27:46,957 - 'RAG 시스템' 검색을 시작합니다...
2024-11-28 11:27:47,558 - 5개의 관련 문서를 찾았습니다.

✨ 검색 완료! 5개의 결과를 찾았습니다.


검색 결과 1/5
제목: 올거나이즈, RAG 성능 평가하는 리더보드 공개..."기업의 RAG 도입에 도움"
날짜: 2024.05.29 17:40
----------------------------------------
URL: https://www.aitimes.com/news/articleView.html?idxno=160144
내용:
기업이 RAG을 도입할 경우 어떤 RAG가 적합한지 성능평가를 제대로 하기 어렵다는 점을 고려, 테스트 데이터셋도 모두 공개했다. 실제 업무 문서에 표, 이미지 등이 복잡하게 얽혀 있다는 점을 감안해 표와 이미지 데이터도 데이터셋에 포함시켰다.
리더보드는 허깅페이스를 통해 확인할 수 있다. 리더보드에 들어간 RAG 솔루션들도 링크로 공개, 누구나 체험해 볼 수 있다.
이창수 올거나이즈 대표는 "AI 생태계에 기여하기 위해 테스트 데이터셋을 모두 공개했으며, 이를 활용해 한국어로 된 다양한 RAG 솔루션의 성능이 향상하길 바란다"라고...

검색 결과 2/5
제목: 인텔리시스, 노코드 RAG 구축 솔루션 ‘레그빌더’ 출시..."RAG 구축 시간 절감"
날짜: 2024.10.23 12:59
----------------------------------------
URL: https://www.aitimes.com/news/articleView.html?idxno=164521
내용:
인텔리시스, 노코드 RAG 구축 솔루션 ‘레그빌더’ 출시..."R

# 하이브리드 검색 시스템

이 코드는 AI 관련 뉴스 기사를 효과적으로 검색하기 위한 하이브리드 검색 시스템을 구현한 것입니다.
벡터 기반 의미론적 검색과 키워드 기반 검색을 결합하여 더 정확한 검색 결과를 제공합니다.

## 주요 기능

1. **문서 처리**
   - JSON 형식의 뉴스 데이터 로드
   - 문서를 청크 단위로 분할
   - 벡터 DB 및 키워드 검색용 인덱스 생성

2. **하이브리드 검색**
   - 벡터 기반 의미론적 검색 (FAISS)
   - 키워드 기반 검색 (BM25)
   - 두 검색 방식의 결과를 가중치를 적용하여 통합

3. **데이터 관리**
   - 벡터 스토어 저장/로드
   - 처리된 문서 데이터 저장/로드
   - 진행 상황 로깅


## 검색 가중치 설정 가이드

- **의미론적 검색 중심 (semantic_weight=0.7)**
  - 문맥과 의미를 더 중요하게 고려
  - 유사한 주제의 문서도 검색 가능
  - 예: "AI 기술의 미래 전망" → AI 발전 방향, 기술 트렌드 등 관련 문서 포함

- **키워드 검색 중심 (semantic_weight=0.3)**
  - 정확한 키워드 매칭을 중시
  - 특정 용어나 개념이 포함된 문서 우선
  - 예: "삼성전자 AI 칩" → 정확히 해당 키워드가 포함된 문서 우선

- **균형잡힌 검색 (semantic_weight=0.5)**
  - 두 방식의 장점을 균형있게 활용
  - 일반적인 검색에 적합
  - 예: "자율주행 안전" → 키워드 매칭과 의미적 연관성 모두 고려

In [ ]:
import os
import json
import glob
import pickle
import logging
import sys
import numpy as np
from typing import List, Dict, Tuple
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

class AINewsRAG:
    """
    AI 뉴스 검색을 위한 RAG(Retrieval-Augmented Generation) 시스템
    
    이 클래스는 뉴스 기사를 벡터 DB로 변환하고, 의미론적 검색과 키워드 기반 검색을
    결합한 하이브리드 검색 기능을 제공합니다.

    Attributes:
        embeddings: OpenAI 임베딩 모델
        text_splitter: 문서 분할을 위한 스플리터
        vector_store: FAISS 벡터 저장소
        bm25: 키워드 기반 검색을 위한 BM25 모델
        processed_docs: 처리된 문서들의 리스트
        doc_mapping: 문서 ID와 문서 객체 간의 매핑
        logger: 로깅을 위한 로거 객체
    """

    def __init__(self, embedding_model):
        """
        AINewsRAG 클래스 초기화

        Args:
            embedding_model: OpenAI 임베딩 모델 인스턴스
        """
        self.embeddings = embedding_model
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        self.vector_store = None
        self.bm25 = None
        self.processed_docs = None
        self.doc_mapping = None
        
        # 로깅 설정
        self.logger = logging.getLogger('AINewsRAG')
        # 기존 핸들러 제거
        if self.logger.handlers:
            self.logger.handlers.clear()
        
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler(sys.stdout)
        handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
        self.logger.addHandler(handler)
        # 로그 중복 방지
        self.logger.propagate = False
        
    def load_json_files(self, directory_path: str) -> List[Dict]:
        """
        여러 JSON 파일에서 뉴스 기사를 로드합니다.

        Args:
            directory_path (str): JSON 파일들이 있는 디렉토리 경로

        Returns:
            List[Dict]: 로드된 뉴스 기사 리스트

        Raises:
            Exception: 파일 로드 중 오류 발생 시
        """
        all_documents = []
        json_files = glob.glob(f"{directory_path}/ai_times_news_*.json")
        
        self.logger.info(f"총 {len(json_files)}개의 JSON 파일을 로드합니다...")
        
        for file_path in tqdm(json_files, desc="JSON 파일 로드 중"):
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    documents = json.load(file)
                    if documents:
                        documents = [doc for doc in documents if len(doc['content']) > 10]
                    if len(documents) >= 10:
                        all_documents.extend(documents)
            except Exception as e:
                self.logger.error(f"파일 로드 중 오류 발생: {file_path} - {str(e)}")
        
        self.logger.info(f"총 {len(all_documents)}개의 뉴스 기사를 로드했습니다.")
        return all_documents
    
    def process_documents(self, documents: List[Dict]) -> List[Document]:
        """문서를 처리하고 청크로 분할합니다."""
        processed_docs = []
        self.logger.info("문서 처리 및 청크 분할을 시작합니다...")
        
        for idx, doc in enumerate(tqdm(documents, desc="문서 처리 중")):
            try:
                #첫 정크에 문서의 제목 포함
                full_text = f"{doc['title']}\n {doc['content']}"
                metadata = {
                    'doc_id': idx, 
                    'title': doc['title'],
                    'url': doc['url'],
                    'date': doc['date']
                }
                
                chunks = self.text_splitter.split_text(full_text)
                
                for chunk_idx, chunk in enumerate(chunks):
                    processed_docs.append(Document(
                        page_content=chunk,
                        metadata={
                            **metadata,
                            'chunk_id': f"doc_{idx}_chunk_{chunk_idx}"  # 청크별 고유 ID
                        }
                    ))
            except Exception as e:
                self.logger.error(f"문서 처리 중 오류 발생: {doc.get('title', 'Unknown')} - {str(e)}")
        
        self.processed_docs = processed_docs
        self.initialize_bm25(processed_docs)
        
        return processed_docs

    def initialize_bm25(self, documents: List[Document]):
        """
        BM25 검색 엔진을 초기화합니다.

        Args:
            documents (List[Document]): 처리된 문서 리스트
        """
        self.logger.info("BM25 검색 엔진을 초기화합니다...")
        
        tokenized_corpus = [
            doc.page_content.lower().split() 
            for doc in documents
        ]
        
        self.bm25 = BM25Okapi(tokenized_corpus)
        self.doc_mapping = {
            i: doc for i, doc in enumerate(documents)
        }
        
        self.logger.info("BM25 검색 엔진 초기화가 완료되었습니다.")
    
    def create_vector_store(self, documents: List[Document]):
        """
        FAISS 벡터 스토어를 생성합니다.

        Args:
            documents (List[Document]): 벡터화할 문서 리스트

        Raises:
            Exception: 벡터 스토어 생성 중 오류 발생 시
        """
        self.logger.info("벡터 스토어 생성을 시작합니다...")
        total_docs = len(documents)
        
        try:
            batch_size = 100
            for i in tqdm(range(0, total_docs, batch_size), desc="벡터 생성 중"):
                batch = documents[i:i+batch_size]
                if self.vector_store is None:
                    self.vector_store = FAISS.from_documents(batch, self.embeddings)
                else:
                    batch_vectorstore = FAISS.from_documents(batch, self.embeddings)
                    self.vector_store.merge_from(batch_vectorstore)
            
            self.logger.info("벡터 스토어 생성이 완료되었습니다.")
        except Exception as e:
            self.logger.error(f"벡터 스토어 생성 중 오류 발생: {str(e)}")
            raise

    def keyword_search(self, query: str, k: int = 5) -> List[Tuple[Document, float]]:
        """
        키워드 기반 BM25 검색을 수행합니다.

        Args:
            query (str): 검색 쿼리
            k (int): 반환할 결과 수

        Returns:
            List[Tuple[Document, float]]: (문서, 점수) 튜플의 리스트

        Raises:
            ValueError: BM25가 초기화되지 않은 경우
        """
        if self.bm25 is None:
            raise ValueError("BM25가 초기화되지 않았습니다.")
        
        self.logger.info(f"'{query}' 키워드 검색을 시작합니다...")
        
        tokenized_query = query.lower().split()
        bm25_scores = self.bm25.get_scores(tokenized_query)
        
        top_k_idx = np.argsort(bm25_scores)[-k:][::-1]
        results = [
            (self.doc_mapping[idx], bm25_scores[idx])
            for idx in top_k_idx
        ]
        
        self.logger.info(f"{len(results)}개의 키워드 검색 결과를 찾았습니다.")
        return results

    def hybrid_search(
            self, 
            query: str, 
            k: int = 5, 
            semantic_weight: float = 0.5
        ) -> List[Tuple[Document, float]]:
            """
            의미론적 검색과 키워드 검색을 결합한 하이브리드 검색을 수행합니다.
            """
            self.logger.info(f"'{query}' 하이브리드 검색을 시작합니다...")
            
            semantic_results = self.vector_store.similarity_search_with_score(query, k=k)
            keyword_results = self.keyword_search(query, k=k)
            
            # 문서 ID를 키로 사용
            combined_scores = {}
            
            # 의미론적 검색 결과 처리
            max_semantic_score = max(score for _, score in semantic_results)
            for doc, score in semantic_results:
                doc_id = doc.metadata['chunk_id']
                
                #5개의 문서의 점수가
                normalized_score = 1 - (score / max_semantic_score) 
                combined_scores[doc_id] = {
                    'doc': doc,
                    'score': semantic_weight * normalized_score
                }
            
            # 키워드 검색 결과 처리
            max_keyword_score = max(score for _, score in keyword_results)
            for doc, score in keyword_results:
                doc_id = doc.metadata['chunk_id']
                normalized_score = score / max_keyword_score
                if doc_id in combined_scores:
                    combined_scores[doc_id]['score'] += (1 - semantic_weight) * normalized_score
                else:
                    combined_scores[doc_id] = {
                        'doc': doc,
                        'score': (1 - semantic_weight) * normalized_score
                    }
            
            # 결과 정렬
            sorted_results = sorted(
                [(info['doc'], info['score']) for info in combined_scores.values()],
                key=lambda x: x[1],
                reverse=True
            )[:k]
            
            self.logger.info(f"{len(sorted_results)}개의 하이브리드 검색 결과를 찾았습니다.")
            return sorted_results

    def save_vector_store(self, vector_store_path: str, processed_docs_path:str=None):
        """
        벡터 스토어와 BM25 데이터를 저장합니다.
        """
        try:
            self.logger.info(f"데이터를 {vector_store_path}에 저장합니다...")
            
            # 벡터 스토어 저장
            os.makedirs(vector_store_path, exist_ok=True)
            self.vector_store.save_local(vector_store_path)
            
            # processed_docs 저장
            if self.processed_docs:
                os.makedirs(os.path.dirname(processed_docs_path), exist_ok=True)
                with open(processed_docs_path, 'wb') as f:
                    pickle.dump(self.processed_docs, f)
            
            self.logger.info("저장이 완료되었습니다.")
        except Exception as e:
            self.logger.error(f"저장 중 오류 발생: {str(e)}")
            raise

    def load_vector_store(self, vector_store_path: str, processed_docs_path):
        """
        벡터 스토어와 BM25 데이터를 로드합니다.
        """
        try:
            self.logger.info(f"데이터를 {vector_store_path}에서 로드합니다...")
            
            # 벡터 스토어 로드
            self.vector_store = FAISS.load_local(
                vector_store_path,
                self.embeddings,
                allow_dangerous_deserialization=True
            )
            
            # processed_docs 로드
            if os.path.exists(processed_docs_path):
                with open(processed_docs_path, 'rb') as f:
                    self.processed_docs = pickle.load(f)
                self.initialize_bm25(self.processed_docs)
            
            self.logger.info("로드가 완료되었습니다.")
        except Exception as e:
            self.logger.error(f"로드 중 오류 발생: {str(e)}")
            raise

### 벡터 스토어 생성

In [3]:
# 환경 변수 로드
from dotenv import load_dotenv
import os
load_dotenv()

# 임베딩 모델 초기화 
embedding_model = OpenAIEmbeddings(
    model=os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
)

# 환경 변수에서 경로 가져오기
vector_store_path = os.getenv("VECTOR_STORE_NAME", "ai_news_vectorstore")
news_dir = os.getenv("NEWS_FILE_PATH", "./ai_news")
processed_doc_path = os.getenv("PROCESSED_DOCS_PATH", "processed_docs/processed_docs.pkl")

# RAG 시스템 초기화
rag = AINewsRAG(embedding_model)

print("새로운 벡터 스토어를 생성합니다...")

# JSON 파일에서 뉴스 데이터 로드
documents = rag.load_json_files(news_dir)

# 문서 처리 및 벡터 스토어 생성
processed_docs = rag.process_documents(documents)
rag.create_vector_store(processed_docs)

# 벡터 스토어 저장
rag.save_vector_store(vector_store_path, processed_doc_path)
print("✅ 새로운 벡터 스토어 생성 및 저장 완료")

c:\Users\tjfan\miniconda3\envs\ai_pjt_02\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


새로운 벡터 스토어를 생성합니다...
2024-11-26 19:41:53,044 - 총 85개의 JSON 파일을 로드합니다...


JSON 파일 로드 중: 100%|██████████| 85/85 [00:00<00:00, 86.88it/s]

2024-11-26 19:41:54,040 - 총 16973개의 뉴스 기사를 로드했습니다.
2024-11-26 19:41:54,042 - 문서 처리 및 청크 분할을 시작합니다...



문서 처리 중: 100%|██████████| 16973/16973 [00:01<00:00, 14529.18it/s]

2024-11-26 19:41:55,214 - BM25 검색 엔진을 초기화합니다...


2024-11-26 19:42:00,913 - BM25 검색 엔진 초기화가 완료되었습니다.
2024-11-26 19:42:01,010 - 벡터 스토어 생성을 시작합니다...


벡터 생성 중:   0%|          | 1/309 [00:06<31:04,  6.05s/it]


KeyboardInterrupt: 

### 하이브리드 서치 테스트

In [4]:

# 환경 변수 로드
from dotenv import load_dotenv
import os
load_dotenv()

# 환경 변수에서 경로 가져오기
vector_store_path = os.getenv("VECTOR_STORE_NAME", "ai_news_vectorstore")
news_dir = os.getenv("NEWS_FILE_PATH", "./ai_news")
processed_doc_path = os.getenv("PROCESSED_DOCS_PATH", "processed_docs/processed_docs.pkl")

# 임베딩 모델 초기화 
embedding_model = OpenAIEmbeddings(
    model=os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
)

# RAG 시스템 초기화
rag = AINewsRAG(embedding_model)

try:
    # 기존 벡터 스토어 로드 시도
    rag.load_vector_store(vector_store_path, processed_doc_path)
    print("✅ 기존 벡터 스토어를 로드했습니다.")
    
except Exception as e:
    print(f"벡터 스토어 로드 실패: {str(e)}")

# 대화형 검색 시작
print("\n🔍 AI 뉴스 검색 시스템을 시작합니다.")
print("- 종료하려면 'q' 또는 'quit'를 입력하세요.")
print("- 검색 방식 변경은 'mode [semantic/keyword/hybrid]'를 입력하세요.")

search_mode = "hybrid"
while True:
    query = input("\n🔍 검색할 내용을 입력하세요: ").strip()

    if not query:
        continue
        
    if query.lower() in ['q', 'quit']:
        print("\n👋 검색을 종료합니다.")
        break
        
    if query.lower().startswith('mode '):
        mode = query.split()[1].lower()
        if mode in ['semantic', 'keyword', 'hybrid']:
            search_mode = mode
            print(f"\n✅ 검색 모드를 '{mode}'로 변경했습니다.")
        else:
            print("\n❌ 잘못된 검색 모드입니다. semantic/keyword/hybrid 중 선택하세요.")
        continue

    try:
        print(f"\n'{query}' 검색을 시작합니다... (모드: {search_mode})")
        
        if search_mode == "hybrid":
            results = rag.hybrid_search(query, k=5, semantic_weight=0.7)
        elif search_mode == "semantic":
            results = rag.vector_store.similarity_search_with_score(query, k=5)
        else:  # keyword
            results = rag.keyword_search(query, k=5)
        
        print(f"\n✨ 검색 완료! {len(results)}개의 결과를 찾았습니다.\n")
        
        # 결과 출력
        for i, (doc, score) in enumerate(results, 1):
            print(f"\n{'='*80}")
            print(f"검색 결과 {i}/{len(results)}")
            print(f"제목: {doc.metadata['title']}")
            print(f"날짜: {doc.metadata['date']}")
            if search_mode == "hybrid":
                print(f"통합 점수: {score:.4f}")
            elif search_mode == "semantic":
                print(f"유사도 점수: {1 - (score/2):.4f}")
            else:
                print(f"BM25 점수: {score:.4f}")
            print(f"URL: {doc.metadata['url']}")
            print(f"{'-'*40}")
            print(f"내용:\n{doc.page_content[:300]}...")
            
    except Exception as e:
        print(f"\n❌ 검색 중 오류가 발생했습니다: {str(e)}")

2024-11-26 19:42:11,832 - 데이터를 ai_news_vectorstore에서 로드합니다...
2024-11-26 19:42:11,942 - 로드 중 오류 발생: '__fields_set__'
벡터 스토어 로드 실패: '__fields_set__'

🔍 AI 뉴스 검색 시스템을 시작합니다.
- 종료하려면 'q' 또는 'quit'를 입력하세요.
- 검색 방식 변경은 'mode [semantic/keyword/hybrid]'를 입력하세요.

👋 검색을 종료합니다.
